In [81]:
# use the sql sent in your emai lto yourself and pull the booking data from BigQuery
import pandas as pd
import numpy as np
from datetime import datetime

# Load and examine data files to understand structure
trans_file = "/Users/vijayaraghavandevaraj/Downloads/WBooking.xlsx"
tax_file = "/Users/vijayaraghavandevaraj/Downloads/Wood - Tax Report.xlsx"

print("📁 Loading Excel files...")
WoodTrans = pd.read_excel(trans_file)
WoodTax = pd.read_excel(tax_file)

print(f"✅ Transaction Report loaded: {len(WoodTrans)} records")
print(f"✅ Tax Report loaded: {len(WoodTax)} records")


📁 Loading Excel files...
✅ Transaction Report loaded: 1136 records
✅ Tax Report loaded: 918 records


In [82]:
#Step 1: Filter the 'trans report' for 'pos' source
trans_pos_df = WoodTrans[WoodTrans['booking_source'].isin(['Website', 'crm'])].copy()
trans_pos_df['order_status'].isin(['booked', 'cancelled','expired']).copy()
trans_pos_df.head()
len(trans_pos_df)

462

In [83]:
# First, let's check what columns are available in trans_pos_df
print("🔍 Checking available columns in trans_pos_df:")
print("Columns:", list(trans_pos_df.columns))
print(f"Shape: {trans_pos_df.shape}")

# Prepare date columns and create Month column
print("\n🔧 Preparing date columns...")

# Convert date columns to datetime
# Use eventDate for trans_pos_df filtering
trans_pos_df['eventDate'] = pd.to_datetime(trans_pos_df['eventDate'])
trans_pos_df['Month'] = trans_pos_df['eventDate'].dt.to_period('M')

print(f"   ✅ Date columns prepared")

# Define target months - you can modify this list to include any months you want
target_months = ["2025-08"]  # August and September 2025
# You can also use: ["2025-08"] for just August, or ["2025-09"] for just September

# Filter trans_pos_df for target months based on eventDate
trans_filtered = trans_pos_df[trans_pos_df['Month'].astype(str).isin(target_months)]

print(f"   • Filtered transactions ({', '.join(target_months)}): {len(trans_filtered):,} records")

# Show breakdown by month
print(f"\n📊 Breakdown by month:")
total_sum = 0
for month in target_months:
    month_data = trans_filtered[trans_filtered['Month'].astype(str) == month]
    trans_count = len(month_data)
    
    # Calculate total amount for the month (looking for 'total' column)
    if 'total' in month_data.columns:
        month_total = month_data['total'].sum()
        total_sum += month_total
        print(f"   • {month}: {trans_count:,} transactions, Total: ${month_total:,.2f}")
    elif 'Total' in month_data.columns:
        month_total = month_data['Total'].sum()
        total_sum += month_total
        print(f"   • {month}: {trans_count:,} transactions, Total: ${month_total:,.2f}")
    elif 'Amount' in month_data.columns:
        month_total = month_data['Amount'].sum()
        total_sum += month_total
        print(f"   • {month}: {trans_count:,} transactions, Amount: ${month_total:,.2f}")
    else:
        print(f"   • {month}: {trans_count:,} transactions")
        # Show available columns that might contain monetary values
        numeric_cols = month_data.select_dtypes(include=[np.number]).columns.tolist()
        if numeric_cols:
            print(f"     Available numeric columns: {numeric_cols}")

# Show overall totals
print(f"\n💰 Overall Totals:")
print(f"   • Total transactions: {len(trans_filtered):,}")
if 'total' in trans_filtered.columns:
    print(f"   • Total sum: ${total_sum:,.2f}")
    print(f"   • Average per transaction: ${total_sum/len(trans_filtered):,.2f}")
elif 'Total' in trans_filtered.columns:
    print(f"   • Total sum: ${total_sum:,.2f}")
    print(f"   • Average per transaction: ${total_sum/len(trans_filtered):,.2f}")
elif 'Amount' in trans_filtered.columns:
    print(f"   • Total amount: ${total_sum:,.2f}")
    print(f"   • Average per transaction: ${total_sum/len(trans_filtered):,.2f}")

# Additional breakdowns by booking type, order_status, and payment_status
print(f"\n📊 Breakdown by Booking Type:")
if 'booking_flow_type' in trans_filtered.columns:
    booking_type_summary = trans_filtered.groupby('booking_flow_type').agg({
        'total': ['count', 'sum'] if 'total' in trans_filtered.columns else ['count'],
        'tax': ['sum'] if 'tax' in trans_filtered.columns else []
    }).round(2)
    print(booking_type_summary)
    
    # Show order status breakdown by booking type
    if 'order_status' in trans_filtered.columns:
        print(f"\n📊 Order Status by Booking Type:")
        order_by_booking = trans_filtered.groupby(['booking_flow_type', 'order_status']).size().unstack(fill_value=0)
        print(order_by_booking)
    
    # Show payment status breakdown by booking type
    if 'payment_status' in trans_filtered.columns:
        print(f"\n📊 Payment Status by Booking Type:")
        payment_by_booking = trans_filtered.groupby(['booking_flow_type', 'payment_status']).size().unstack(fill_value=0)
        print(payment_by_booking)
        
else:
    print("   • booking_flow_type column not found")

# Tax Analysis
print(f"\n💰 Tax Analysis:")
if 'tax' in trans_filtered.columns:
    total_tax = trans_filtered['tax'].sum()
    print(f"   • Total tax collected: ${total_tax:,.2f}")
    print(f"   • Tax as % of total revenue: {(total_tax/total_sum)*100:.2f}%")
else:
    print("   • tax column not found")


🔍 Checking available columns in trans_pos_df:
Columns: ['order_id', 'payment_status', 'order_status', 'booking_flow_type', 'tip', 'discount', 'Location_Name', 'order_created_date', 'eventDate', 'total', 'tax', 'amountDue', 'amountPaid', 'booking_source']
Shape: (462, 14)

🔧 Preparing date columns...
   ✅ Date columns prepared
   • Filtered transactions (2025-08): 202 records

📊 Breakdown by month:
   • 2025-08: 202 transactions, Total: $5,488.25

💰 Overall Totals:
   • Total transactions: 202
   • Total sum: $5,488.25
   • Average per transaction: $27.17

📊 Breakdown by Booking Type:
                  total              tax
                  count      sum     sum
booking_flow_type                       
party                 1   603.63   53.63
special-event         1    76.83    6.83
time-slot           200  4807.79  426.79

📊 Order Status by Booking Type:
order_status       booked
booking_flow_type        
party                   1
special-event           1
time-slot             200


In [ ]:
# Generate Detailed PDF Report
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from reportlab.platypus.flowables import KeepTogether
from reportlab.graphics.shapes import Drawing
from reportlab.graphics.charts.textlabels import Label
import datetime

def create_rotated_table(table_data, col_widths, table_style):
    """Create a rotated table for better fit"""
    # For wide tables, we'll create them with smaller column widths
    # and use a more compact layout
    rotated_table = Table(table_data, colWidths=col_widths)
    rotated_table.setStyle(table_style)
    return rotated_table

def generate_booking_report_pdf():
    # Create filename with timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Booking_Report_August_2025_{timestamp}.pdf"
    
    # Create PDF document in landscape mode
    from reportlab.lib.pagesizes import landscape, A4
    doc = SimpleDocTemplate(filename, pagesize=landscape(A4), 
                          rightMargin=50, leftMargin=50, 
                          topMargin=72, bottomMargin=18)
    
    # Container for the 'Flowable' objects
    elements = []
    
    # Define professional color scheme
    dark_ash = colors.Color(0.3, 0.3, 0.3)  # Dark ash color
    white = colors.white
    
    # Get styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=18,
        spaceAfter=30,
        alignment=TA_CENTER,
        textColor=dark_ash
    )
    
    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        fontSize=14,
        spaceAfter=12,
        textColor=dark_ash
    )
    
    # Title
    title = Paragraph("Woodland Play Cafe - Booking Analysis Report", title_style)
    elements.append(title)
    
    # Report metadata
    report_date = datetime.datetime.now().strftime("%B %d, %Y")
    elements.append(Paragraph(f"<b>Report Date:</b> {report_date}", styles['Normal']))
    elements.append(Paragraph(f"<b>Analysis Period:</b> August 2025", styles['Normal']))
    elements.append(Paragraph(f"<b>Total Records Analyzed:</b> {len(trans_filtered):,}", styles['Normal']))
    elements.append(Spacer(1, 20))
    
    # Executive Summary
    elements.append(Paragraph("Executive Summary", heading_style))
    
    # Calculate key metrics
    total_transactions = len(trans_filtered)
    total_amount = trans_filtered['total'].sum() if 'total' in trans_filtered.columns else 0
    total_tax = trans_filtered['tax'].sum() if 'tax' in trans_filtered.columns else 0
    avg_transaction = total_amount / total_transactions if total_transactions > 0 else 0
    tax_percentage = (total_tax / total_amount * 100) if total_amount > 0 else 0
    
    summary_data = [
        ['Metric', 'Value'],
        ['Total Transactions', f"{total_transactions:,}"],
        ['Total Revenue', f"${total_amount:,.2f}"],
        ['Total Tax Collected', f"${total_tax:,.2f}"],
        ['Average Transaction Value', f"${avg_transaction:,.2f}"],
        ['Analysis Period', 'August 2025']
    ]
    
    summary_table = Table(summary_data, colWidths=[3*inch, 2.5*inch])
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), dark_ash),
        ('TEXTCOLOR', (0, 0), (-1, 0), white),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), white),
        ('GRID', (0, 0), (-1, -1), 1, dark_ash)
    ]))
    
    elements.append(summary_table)
    elements.append(Spacer(1, 20))
    
    # Order Status by Booking Type
    if 'order_status' in trans_filtered.columns:
            elements.append(Paragraph("Order Status by Booking Type", heading_style))
            
            order_by_booking = trans_filtered.groupby(['booking_flow_type', 'order_status']).size().unstack(fill_value=0)
            
            # Create a more compact table by transposing the data
            # This makes it vertical instead of horizontal
            compact_data = []
            
            # Add header row
            header = ['Booking Type']
            for status in order_by_booking.columns:
                header.append(status)
            compact_data.append(header)
            
            # Add data rows
            for booking_type in order_by_booking.index:
                row = [str(booking_type)]
                for status in order_by_booking.columns:
                    row.append(f"{int(order_by_booking.loc[booking_type, status]):,}")
                compact_data.append(row)
            
            # Use compact column widths to keep table on left side
            col_widths = [1.8*inch] + [1.2*inch] * len(order_by_booking.columns)
            order_table = Table(compact_data, colWidths=col_widths)
            order_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), dark_ash),
                ('TEXTCOLOR', (0, 0), (-1, 0), white),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 9),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
                ('BACKGROUND', (0, 1), (-1, -1), white),
                ('GRID', (0, 0), (-1, -1), 1, dark_ash)
            ]))
            
            elements.append(order_table)
            elements.append(Spacer(1, 10))
        
    # Add page break before Tax Analysis
    elements.append(PageBreak())
    
    # Tax Analysis Section
    if 'tax' in trans_filtered.columns:
        elements.append(Paragraph("Tax Analysis", heading_style))
        
        # Calculate tax metrics
        total_tax = trans_filtered['tax'].sum()
        tax_percentage = (total_tax / total_amount * 100) if total_amount > 0 else 0
        
        # Tax breakdown by booking type
        if 'booking_flow_type' in trans_filtered.columns:
            tax_breakdown_data = trans_filtered.groupby('booking_flow_type').agg({
                'tax': ['count', 'sum']
            }).round(2)
            
            # Prepare tax table data
            tax_table_data = [['Booking Type', 'Count', 'Tax Amount']]
            for idx, row in tax_breakdown_data.iterrows():
                tax_table_data.append([
                    str(idx),
                    f"{int(row[('tax', 'count')]):,}",
                    f"${row[('tax', 'sum')]:,.2f}"
                ])
            
            tax_table = Table(tax_table_data, colWidths=[2.8*inch, 1.5*inch, 2*inch])
            tax_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), dark_ash),
                ('TEXTCOLOR', (0, 0), (-1, 0), white),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 10),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), white),
                ('GRID', (0, 0), (-1, -1), 1, dark_ash)
            ]))
            
            
            elements.append(tax_table)
            elements.append(Spacer(1, 10))
        
    # Add page break before detailed records
    elements.append(PageBreak())
    
    # Detailed Records Section
    elements.append(Paragraph("Detailed Transaction Records", heading_style))
    elements.append(Paragraph(f"This section contains all {len(trans_filtered):,} transaction records for reference.", styles['Normal']))
    elements.append(Spacer(1, 10))
    
    # Prepare data for the detailed records table
    # Select key columns to display (avoid too many columns for readability)
    key_columns = []
    if 'order_id' in trans_filtered.columns:
        key_columns.append('order_id')
    if 'eventDate' in trans_filtered.columns:
        key_columns.append('eventDate')
    if 'booking_flow_type' in trans_filtered.columns:
        key_columns.append('booking_flow_type')
    if 'order_status' in trans_filtered.columns:
        key_columns.append('order_status')
    if 'payment_status' in trans_filtered.columns:
        key_columns.append('payment_status')
    if 'total' in trans_filtered.columns:
        key_columns.append('total')
    if 'tax' in trans_filtered.columns:
        key_columns.append('tax')
    if 'Location_Name' in trans_filtered.columns:
        key_columns.append('Location_Name')
    
    # Create detailed records table
    if key_columns:
        # Prepare table data with serial numbers
        # Add 'S.No' as the first column
        header_row = ['S.No'] + key_columns
        records_data = [header_row]  # Header row with serial number
        
        # Add ALL data rows (not just first 50)
        for index, (_, row) in enumerate(trans_filtered[key_columns].iterrows(), 1):
            row_data = [str(index)]  # Serial number starting from 1
            for col in key_columns:
                value = str(row[col])
                # Truncate long values for better display
                if len(value) > 20:
                    value = value[:17] + "..."
                row_data.append(value)
            records_data.append(row_data)
        
        # Create table with optimized column widths for landscape layout
        # Serial number column is narrower, other columns optimized for landscape
        col_widths = [0.7*inch] + [1.2*inch] * len(key_columns)  # S.No column + data columns
        records_table = Table(records_data, colWidths=col_widths)
        
        # Style the table
        records_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), dark_ash),
            ('TEXTCOLOR', (0, 0), (-1, 0), white),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 8),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), white),
                            ('FONTSIZE', (0, 1), (-1, -1), 8),
            ('GRID', (0, 0), (-1, -1), 0.5, dark_ash),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [white, white])
        ]))
        
        elements.append(records_table)
        
        # Add note about total records
        elements.append(Spacer(1, 10))
        elements.append(Paragraph(f"<i>Note: Showing all {len(trans_filtered):,} transaction records with serial numbers for complete reference.</i>", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    print(f"✅ PDF report generated successfully: {filename}")
    print(f"📄 Report contains comprehensive analysis of {len(trans_filtered):,} transactions")
    
    return filename

# Generate the PDF report
pdf_filename = generate_booking_report_pdf()


✅ PDF report generated successfully: Booking_Report_August_2025_20250915_193654.pdf
📄 Report contains comprehensive analysis of 202 transactions
